In [1]:
import pandas as pd
import datasets
from collections import Counter
import ast
import json

all_human_annotations = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations.csv')

In [2]:
all_human_annotations


,review_point,paper_id,venue,focused_review,actionability,actionability_label,actionability_label_type,batch,grounding_specificity,grounding_specificity_label,...,helpfulness_label_type,professional_tone,professional_tone_label,professional_tone_label_type,valid_point,valid_point_label,valid_point_label_type,addressed_to_author,addressed_to_author_label,addressed_to_author_label_type
0,- There were too many missing details (for exa...,ARR_2022_60_review,ARR_2022,- Underdefined and conflation of concepts - Se...,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,1,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,...,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
1,"- Section 4.3, model parameters and training. ...",ACL_2017_333_review,ACL_2017,There are some few details on the implementati...,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,1,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,...,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
2,- The differences in results in Table 2 are ve...,ARR_2022_269_review,ARR_2022,- It is not clear for me about the novelty of ...,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,1,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,...,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': ['DVRTnFRi', 'boda'], 'labels':...",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
3,"- While the language has been improved, there ...",ARR_2022_331_review,ARR_2022,"- While the language has been improved, there ...","{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,1,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,...,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
4,1) The character tri-gram LSTM seems a little ...,ACL_2017_477_review,ACL_2017,1) The character tri-gram LSTM seems a little ...,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,1,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,...,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': ['fZav2G06', 'DVRTnFRi', 'boda'...",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,"2021 Oct;27(4):710-36. Yes, the author(s) do b...",NIPS_2022_874,NIPS_2022,I found the presentation at times to be more c...,"{'annotators': ['xT364YSG', 'KfBeSZkz'], 'labe...",NaN,NaN,9,"{'annotators': ['xT364YSG', 'KfBeSZkz'], 'labe...",NaN,...,NaN,"{'annotators': ['xT364YSG', 'KfBeSZkz'], 'labe...",NaN,NaN,"{'annotators': ['xT364YSG', 'KfBeSZkz'], 'labe...",NaN,NaN,"{'annotators': ['xT364YSG', 'KfBeSZkz'], 'labe...",NaN,NaN
1331,- I’m missing comparison with a NeRF-based met...,w5oP27fmYW,ICLR_2024,- Main concern: While the improvement in resul...,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,9,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,...,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN
1332,"- Line 14, 47: A brief explanation of “multi-a...",NIPS_2021_1604,NIPS_2021,).\nWeaknesses - Some parts of the paper are d...,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,9,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,...,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN,"{'annotators': ['boda', 'xT364YSG', 'KfBeSZkz'...",NaN,NaN
1333,"3. p indicates the proportion of documents, I ...",ARR_2022_108_review,ARR_2022,"1. First of all, compared 

In [3]:
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', "professional_tone", 'valid_point', 'addressed_to_author']
processed_data = []

for index, row in all_human_annotations.iterrows():
    for aspect in aspects:
        ## get the most common label
        ## load the labels as json
        labels = row[aspect].replace("'", "\"")
        labels = json.loads(labels)['labels']
        if labels:
            # labels = ast.literal_eval(labels)
            frequency = Counter(labels)
            most_frequent_value, occurrence = frequency.most_common(1)[0]
        else:
            most_frequent_value = 'None'
            occurrence = 0
        if occurrence > 1:
            row[f'{aspect}_label'] = most_frequent_value
            if occurrence > 2:
                row[f'{aspect}_label_type'] = 'gold'
            else:
                row[f'{aspect}_label_type'] = 'silver'
        else:
            row[f'{aspect}_label'] = 'None'
            row[f'{aspect}_label_type'] = 'None'
    processed_data.append(row)

processed_data = pd.DataFrame(processed_data)



In [4]:
processed_data.to_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv', index=False)

In [5]:
print(len(processed_data))

1335


In [6]:
import krippendorff
import numpy as np
## group entries by each batch number, and then output the statiscs for each aspect also write it to txt file
with open('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/annotation_statistics.txt', 'w') as f:
    for batch in range(1, 10):
        print(f'Batch: {batch}')
        f.write(f'Batch: {batch}\n')
        batch_data = processed_data[processed_data['batch'] == batch]
        print(len(batch_data))
        f.write(f'Number of entries: {len(batch_data)}\n')
        for aspect in aspects:
            print(f'Aspect: {aspect}')
            f.write(f'Aspect: {aspect}\n')
            print(batch_data[f'{aspect}_label_type'].value_counts())
            f.write(f'{batch_data[f"{aspect}_label_type"].value_counts()}\n')
            print('---------------------------------')
            f.write('---------------------------------\n')

    ######### total statistics
    print('Total Statistics')
    f.write('Total Statistics\n')
    print(len(processed_data))
    f.write(f'Number of entries: {len(processed_data)}\n')
    for aspect in aspects:
        # if aspect == 'verifiability':
        #     continue
        ## measure Krippendorff's alpha
        # final_data = []
        # for index, row in processed_data.iterrows():
        #     row = row[aspect]
        #     row = ast.literal_eval(row)
        #     # if one valye is X make it 0
        #     row = [0 if r == 'X' else r for r in row]
        #     ## convert it to int
        #     row = [int(r) for r in row]
        #     if len(row) >= 3:
        #         final_data.append(row[:3])

        # final_data = np.array(final_data).T
       
        # # print(f'Krippendorff alpha for {aspect}: {krippendorff.alpha(final_data,)}')
        # # f.write(f'Krippendorff alpha for {aspect}: {krippendorff.alpha(final_data)}\n')

        print(f'Aspect: {aspect}')
        f.write(f'Aspect: {aspect}\n')
        print(processed_data[f'{aspect}_label_type'].value_counts())
        f.write(f'{processed_data[f"{aspect}_label_type"].value_counts()}\n')
        print('---------------------------------')
        f.write('---------------------------------\n')


Batch: 1
25
Aspect: actionability
actionability_label_type
silver    14
None       7
gold       4
Name: count, dtype: int64
---------------------------------
Aspect: grounding_specificity
grounding_specificity_label_type
silver    13
None       7
gold       5
Name: count, dtype: int64
---------------------------------
Aspect: verifiability
verifiability_label_type
silver    14
None      10
gold       1
Name: count, dtype: int64
---------------------------------
Aspect: helpfulness
helpfulness_label_type
None      12
silver    11
gold       2
Name: count, dtype: int64
---------------------------------
Aspect: professional_tone
professional_tone_label_type
silver    13
gold      12
Name: count, dtype: int64
---------------------------------
Aspect: valid_point
valid_point_label_type
silver    21
gold       3
None       1
Name: count, dtype: int64
---------------------------------
Aspect: addressed_to_author
addressed_to_author_label_type
None    25
Name: count, dtype: int64
-------------